In [1]:
'''
==========
Exercise 2
==========

Association rules learning algorithm for related queries:
---------------------------------------------------------

input: list of pairs of (user id, search query)
output: For query Sx: {Sx} ==> {Sy} with confidence Cxy

Definitions:
------------

n - number of users
L[i] - number of queries of user i
1<=i<=n
1<=j<=L[i]
r - total number of queries
r = sigma(L[i]) [i=1..n]
1<=k,k1,k2<=r

u[i] - user i
q[i,j] - query j of user i
count(q) - count number of occurences of q in the data set
count(q[k1] V q[k2]) - count number of occurences of query q[k1] and query q[k2] in the data set

Algorithm:
----------

Supp(x) = |{tϵT; x ⊆ t}| / (|T|)

Conf(x V y) = Supp(y V x) / Supp(x) = |{tϵT; x,y ⊆ t}| / |{tϵT; x ⊆ t}|


Algorithm implementation high level:
------------------------------------
1. Build users_queries_df, which consist of (u[i], q[i,j])
2. Build user_queries_count_df, which consist of (q[i,j], u[i], count(q[i,j]))
3. Build queries_count_df, that consist of (q[k], count(q[k]))
4. Build users_pair_queries_count_df, which consist of ((q[k1], q[k2]), count(q[k1] V q[k2]))
5. Build results_df, which consist of (q[k1], q[k2], count(q[k1] V q[k2]), count(q[k1])])
6. Add statistics & math columns to results_df
7. Display results
'''

'\n==========\nExercise 2\n==========\n\nAssociation rules learning algorithm for related queries:\n---------------------------------------------------------\n\ninput: list of pairs of (user id, search query)\noutput: For query Sx: {Sx} ==> {Sy} with confidence Cxy\n\nDefinitions:\n------------\n\nn - number of users\nL[i] - number of queries of user i\n1<=i<=n\n1<=j<=L[i]\nr - total number of queries\nr = sigma(L[i]) [i=1..n]\n1<=k,k1,k2<=r\n\nu[i] - user i\nq[i,j] - query j of user i\ncount(q) - count number of occurences of q in the data set\ncount(q[k1] V q[k2]) - count number of occurences of query q[k1] and query q[k2] in the data set\n\nAlgorithm:\n----------\n\nSupp(x) = |{tϵT; x ⊆ t}| / (|T|)\n\nConf(x V y) = Supp(y V x) / Supp(x) = |{tϵT; x,y ⊆ t}| / |{tϵT; x ⊆ t}|\n\n\nAlgorithm implementation high level:\n------------------------------------\n1. Build users_queries_df, which consist of (u[i], q[i,j])\n2. Build user_queries_count_df, which consist of (q[i,j], u[i], count(q[i

In [2]:
from pyspark.context import SparkContext # required for reading input dataset & stop queries.
from pyspark.sql.functions import count, col, length, udf, pow, lit, broadcast
# count is required for aggregating queries, by number of occurences in the data set.
# col is required for filtering out stop websites,  selecteing columns in data frames filters.
# length is required for filtering out queries with small length.
# udf is required for adding columns to results data frame
# pow is required for calculating levenshtein threshold of confidence column
# lit is required for calling the function that calculates levenshtein distance threshold of confidence column
# broadcast is required for the removal of stop queries, which includes, using join between 2 data frames.
from pyspark.sql.types import IntegerType, FloatType
# IntegerType is required for adding confidence_level & similarity columns to results data frame, using udf
# FloatType is required for adding confidence column to results data frame, using udf
from pyspark.sql import Row # required for converting pair of queries, and their count, from rdd to data frame
import math # required for calculating the floor for: 1) minimum number of queries pair. 2) confidence level
from itertools import combinations # Required for creating combinations of queries for each user
from operator import add # Required in reduce operation, for counting number of occurences of queries pairs.

In [3]:
print("Define global variables")
# min num of rows to display
n1 = 10

# max num of rows to display
n2 = 20

# Sorted list of confidences
confidences = [0.6, 0.8, 0.9, 1]

# Minimum number of characters in each query.
min_num_of_chars_in_query = 2

# Minimum number of occurences of a aingle query.
min_num_of_queries = 6

# Minimum number of occurences of pair of queries for different users.
min_num_of_queries_pair =  math.floor(min_num_of_queries * confidences[0])

# threshold for similarity between 2 queries
levenshtein_distance_threshold = 10

# threshold for collecting results as list (in order to display the results on the console in a readable format.)
display_rules_num_of_records_threshold = 400

# Irrlevant websites, that probably not related directly to search queries.
stop_websites = 'google|gmail|mapquest|ebay|myspace|yahoo'

Define global variables


In [4]:
'''
Algorithm implementation, in details:
-------------------------------------

0. Read input data
    0.1 Read input users_queries_search_main_df with headers
    0.2 Read stop_queries_df without headers

1. Build users_queries_df, which consist of (u[i], q[i,j])
    1.1 select only user & query columns, from users_queries_search_main_df, and set it into users_queries_df
    1.1 filter out queries that contains stop websites
    1.2 filter out queries with length less than min_num_of_chars_in_query
    1.3 remove duplicate entries
    1.4 filter out stop queries (related to the whole query, rather than string inside the query)
        1.4.1 set stop_queries_broadcast to be the broadcast of stop_queries_df
        1.4.2 set unwanted_queries_df to be the join of users_queries_df and stop_queries_broadcast, by query column
        1.4.3 users_queries_df = users_queries_df - unwanted_queries_df

2. Build user_queries_count_df, which consist of (q[i,j], u[i], count(q[i,j]))
    2.1 join users_queries_df with queries_count_df, by query column
    
3. Build queries_count_df, that consist of (q[k], count(q[k]))
    3.1 aggregate group user_queries_count_df by query, and count queries
    3.2 filter out queries with number of occurences less than min_num_of_queries
    3.3 remove duplicate entries of ('query', 'count_query')

4. Build users_pair_queries_count_df, which consist of ((q[k1], q[k2]), count(q[k1] V q[k2]))
    by performing the following operations:
    4.1 map: map user to suitable queries. (query, user, count_query) ==> (user, query)
    4.2 reduceByKey: For each user add the suitable list of queries. ((u1, [q1,q2,...]), (u2, [q2,q5,...]), ...)
    4.3 map: create pairs of queries combinations, for each user & drop users. ((q1,q2),(q1,q5),(q2,q5)...)
    4.4 flatMap add 1 to each query pair ((q1,q2),(q1,q5),(q2,q5)...) ==> (((q1,q2),1),((q1,q5),1),((q2,q5),1)...)
    4.5 reduceByKey count num of occurences of each query pair (((q1,q2),3),((q1,q5),7),((q2,q5),102)...)
    4.6 filter filter out queries, with number of occurences below min_num_of_queries_pair.
        For example: if min_num_of_queries_pair = 6, then the tuple ((q1,q2),3) will be filtered out.

5. Build results_df, which consist of (q[k1], q[k2], count(q[k1] V q[k2]), count(q[k1])])
    5.1 Build query_results_df, that consist of (q[k1], q[k2], count(q[k1] V q[k2]), count(q[k1])])
        5.1.1 join users_pair_queries_count_df with queries_count_df, by query column
        5.1.2 filter queries with confidence less than the minimum confidence (which is confidences[0])
    5.2 Build query2_results_df, that consist of (q[k2], q[k1], count(q[k1] V q[k2]), count(q[k2])])
        5,2,1 join users_pair_queries_count_df with queries_count_df, by query column (rename query to query2)
        5.2.2 filter queries with confidence less than the minimum confidence (which is confidences[0])
    5.3 results_df is the union of query_results_df and query2_results_df, based on the key column (the first column)

6. Add statistics & math columns to results_df
    6.1 Add confidence column:
        6.1.1 confidence(q[k1], q[k2]) = count(q[k1] V q[k2]) / count(q[k1])
    6.2 Add confidence_level column:
        6.2.1 confidence_level = sigma(floor(conf * 10) / floor(confidences[i] * 10)), i=0.. len(confidences)
        In case of confidences = [0.6, 0.8, 0.9, 1], we can simplify the definition to:
        confidence_level(confidence) = {
            1, if confidence = 1
            2, if confidence in range [0.9,1)
            3, if confidence in range [0.8,0.9)
            4, if confidence in range [0.6,0.8)
        }
    6.3 Add similarity column:
        6.3.1 similarity = levenshtein(q[k1], q[k2])
    6.4 Filter out queries with high similrity (high similrity <==> low scale of the levenshtein score) and low confidence
        6.4.1 filter out queries if:
                levenshtein(q[k1], q[k2]) < distance_threshhold * (1- pow(confidence(q[k1], q[k2]), 2))

7. Display results
    7.1 Save results_df to disk partioned by confidence level
    7.2 Display results on the screen
        7.2.1 If number of results is less than display_rules_num_of_records_threshhold, then:
                Collect results as list and display it in the following format:
                q[k1] ==> q[k2], conf=[confidence(q[k1], q[k2])], #q1=[count(q[k1])], #(q1 and q2)=[count(q[k1] V q[k2])], similarity=[similarity(q[k1], q[k2])]
        7.2.2 Else: print results_df

'''

"\nAlgorithm implementation, in details:\n-------------------------------------\n\n0. Read input data\n    0.1 Read input users_queries_search_main_df with headers\n    0.2 Read stop_queries_df without headers\n\n1. Build users_queries_df, which consist of (u[i], q[i,j])\n    1.1 select only user & query columns, from users_queries_search_main_df, and set it into users_queries_df\n    1.1 filter out queries that contains stop websites\n    1.2 filter out queries with length less than min_num_of_chars_in_query\n    1.3 remove duplicate entries\n    1.4 filter out stop queries (related to the whole query, rather than string inside the query)\n        1.4.1 set stop_queries_broadcast to be the broadcast of stop_queries_df\n        1.4.2 set unwanted_queries_df to be the join of users_queries_df and stop_queries_broadcast, by query column\n        1.4.3 users_queries_df = users_queries_df - unwanted_queries_df\n\n2. Build user_queries_count_df, which consist of (q[i,j], u[i], count(q[i,j])

In [5]:
print("Load the data set")
users_queries_search_main_df = spark.read.option("header", "true") \
    .option("delimiter", "\t") \
    .csv("user-ct-test-collection-01.txt")

print("Load stop queries")
stop_queries_df = spark.read.option("header", "false")\
    .option("delimiter", "\n")\
    .csv("stop-queries.txt")

Load the data set
Load stop queries


In [6]:
print("Filter out queries that contains words from stop_websites or \
queries with length below " + repr(min_num_of_chars_in_query) + " characters.")
users_queries_filter_condition = (col('Query').rlike(stop_websites) == False) & \
                                (length(col("Query")) >= min_num_of_chars_in_query)

print("Get data frame of (user, query), with the above filter & frop duplicates of (user, query)")
users_queries_df = users_queries_search_main_df.select('AnonID', 'Query')\
                    .drop_duplicates(subset=['AnonID', 'Query'])\
                    .filter(users_queries_filter_condition)\
                    .select(col('AnonID').alias('user'), col('Query').alias('query'))

print("create a broadcast from stop_queries_df")
stop_queries_broadcast = broadcast(stop_queries_df.select(col('_c0').alias('query')))
                                   
print("Join users_queries_df with stop_queries_broadcast, by the query column")
unwanted_queries_df = users_queries_df.join(stop_queries_broadcast, on='query' , how = 'inner').select('user', 'query')
users_queries_df = users_queries_df.subtract(unwanted_queries_df)

print("repartition users_queries_df by user column")
users_queries_df.repartition('user')

print("Get query count data frame, and filter out queries with number of occurences below " + repr(min_num_of_queries))
queries_count_df = users_queries_df.groupBy('query').agg(count("*").alias("count_query"))\
                    .filter("count_query >= " + repr(min_num_of_queries))\
                    .drop_duplicates(subset=['query', 'count_query'])               

print("repartition queries_count_df by query column")
queries_count_df.repartition('query')

print("Join users_queries_df with users_queries_df, by query column, to get a data frame of (query, user, count_query)")
users_queries_count_df = users_queries_df.join(queries_count_df, on='query', how='inner')

Filter out queries that contains words from stop_websites or queries with length below 2 characters.
Get data frame of (user, query), with the above filter & frop duplicates of (user, query)
create a broadcast from stop_queries_df
Join users_queries_df with stop_queries_broadcast, by the query column
repartition users_queries_df by user column
Get query count data frame, and filter out queries with number of occurences below 6
repartition queries_count_df by query column
Join users_queries_df with users_queries_df, by query column, to get a data frame of (query, user, count_query)


In [7]:
'''
users_pair_queries_count_df is converted to rdd,
in order to avoid self cartesian join (which is required in order to get all of the combinations of pair of queries)
by performing the following operations:
.map: map user to suitable queries. (query, user, count_query) ==> (user, query)
.reduceByKey: For each user add the suitable list of queries. ((u1, [q1,q2,...]), (u2, [q2,q5,...]), ...)
.map: create pairs of queries combinations, for each user & drop users. ((q1,q2),(q1,q5),(q2,q5)...)
.flatMap add 1 to each query pair ((q1,q2),(q1,q5),(q2,q5)...) ==> (((q1,q2),1),((q1,q5),1),((q2,q5),1)...)
.reduceByKey count num of occurences of each query pair (((q1,q2),3),((q1,q5),7),((q2,q5),102)...)
.filter filter out queries, with number of occurences below min_num_of_queries_pair.
        For example: if min_num_of_queries_pair = 6, then the tuple ((q1,q2),3) will be filtered out.
'''
users_pair_queries_count_rdd = users_queries_count_df.rdd\
                        .map(lambda line: (line[1], [line[0]]))\
                        .reduceByKey(add)\
                        .map(lambda line: tuple(combinations(line[1], 2)))\
                        .flatMap(lambda line: [(x, 1) for x in line])\
                        .reduceByKey(add)\
                        .filter(lambda line: line[1] >= min_num_of_queries_pair)

print("Convert users_pair_queries_count_rdd to data frame, in order to join it with queries_count_df")
users_pair_queries_count_df = sqlContext.createDataFrame(users_pair_queries_count_rdd.map(lambda line: Row(query=line[0][0], query2=line[0][1], count_2_queries=line[1])))

Convert users_pair_queries_count_rdd to data frame, in order to join it with queries_count_df


In [8]:
print("Join query count to results data frame")
query_results_df = users_pair_queries_count_df.join(queries_count_df, on = 'query', how = 'inner')\
                                        .filter('count_2_queries / count_query >= ' + repr(confidences[0]))\
                                        .select('query', 'query2', 'count_2_queries', 'count_query')

query2_results_df = users_pair_queries_count_df.join(queries_count_df.select(col('query').alias('query2'), col('count_query')), on = 'query2', how = 'inner')\
                                        .filter('count_2_queries / count_query >= ' + repr(confidences[0]))\
                                        .select('query', 'query2', 'count_2_queries', 'count_query')

results_df = query_results_df.union(query2_results_df)

Join query count to results data frame


In [9]:
# Get similarity between 2 strings.
#
# Parameters:
# s - first string
# t - second string
#
# The similarity between s and t is,
# the minimum number of single-character edits (i.e. insertions, deletions, or substitutions),
# required to change one word into the other.
# Therefore low number of edit actions means high similarirty between s and t, and vice versa.
def levenshtein(s, t):
        ''' From Wikipedia article; Iterative with two matrix rows. '''
        if s == t: return 0
        elif len(s) == 0: return len(t)
        elif len(t) == 0: return len(s)
        v0 = [None] * (len(t) + 1)
        v1 = [None] * (len(t) + 1)
        for i in range(len(v0)):
            v0[i] = i
        for i in range(len(s)):
            v1[0] = i + 1
            for j in range(len(t)):
                cost = 0 if s[i] == t[j] else 1
                v1[j + 1] = min(v1[j] + 1, v0[j + 1] + 1, v0[j] + cost)
            for j in range(len(v0)):
                v0[j] = v1[j]
                
        return v1[len(t)]

# Get normalized levenshtein threshold, based on initial threshold and confidence.
#
# Parameters :
# distance_threshold - initial levenshtein distance threshold
# confidence - the confidence that 2 queries: q1,q2 are associated: q1 ==> q2
#
# The  levenshtein threshold is required for filtering queries, based on low similarity, and high confidence.
# We would like to get results with confidence as high as possible, and similarity as low as possible.
# Therefore for high confidence we can compromise the demand for low similarity, and vice versa.
# This threshold should be compared with the similarity of 2 queries.
# The requirement is to have a similarity no less than this threshold
# Low similarity (high levenshtein value) will be accepted only with high confidence (and vice versa).
def levenshtein_threshold(distance_threshold, confidence):
    return distance_threshold * (1- pow(confidence, 2))

# Calculate confidence of 2 queries.
#
# Parameters:
# count_2_queries - number of occurences of 2 queries: q1, q2
# count_query - number of occurences of a single query: q1
def conf(count_2_queries, count_query):
    return count_2_queries / count_query

# partition the possible confidences to groups.
# The ammount of groups, is the size of confidences list
# conf_level = sigma(floor(conf * 10) / floor(confidences[i] * 10)), i=0.. len(confidences)
def conf_level(conf):
    factor = math.floor(conf * 10)
    l = len(confidences)
    conf_level = 0
    i = 0
    while(i < l and conf >= confidences[i]):
        conf_level += math.floor(factor / math.floor(confidences[i] * 10))
        i += 1
    return conf_level

In [10]:
print("Add confidence column to result data frame")
func_conf_udf = udf(conf, FloatType())
results_df = results_df.withColumn('conf',func_conf_udf(results_df['count_2_queries'], results_df['count_query']))

print("Add confidence_level column to result data frame")
func_conf_level_udf = udf(conf_level, IntegerType())
results_df = results_df.withColumn('conf_level',func_conf_level_udf(results_df['conf']))

print("Add similarity column to result data frame")
func_levenshtein_udf = udf(levenshtein, IntegerType())
results_df = results_df.withColumn('similarity',func_levenshtein_udf(results_df['query'], results_df['query2']))

print("Filter queries with high similarity, using levenshtein algorithm")
results_df = results_df.filter(results_df['similarity'] >= levenshtein_threshold(levenshtein_distance_threshold, lit(results_df['conf'])))

print("Sort results")
results_df = results_df.orderBy(['conf_level', 'conf', 'similarity', 'count_2_queries', 'count_query'], ascending=False)

print("repartition results_df by conf_level column")
results_df.repartition('conf_level')

print("Count number of results")
num_of_results = results_df.count()
print('num of results = ' + repr(num_of_results))

Add confidence column to result data frame
Add confidence_level column to result data frame
Add similarity column to result data frame
Filter queries with high similarity, using levenshtein algorithm
Sort results
repartition results_df by conf_level column
Count number of results
num of results = 58


In [11]:
folder_name = 'related_searches'
print('Save results data frame as ' + folder_name)
results_df.coalesce(1).write\
            .partitionBy('conf_level')\
            .format("com.databricks.spark.csv")\
            .option("header", "true")\
            .mode("overwrite")\
            .save(folder_name)

Save results data frame as related_searches


In [12]:
# Display a given character for a given length times
def get_display_multiple_characters(character, title, length):
    return character * length

# Display title, with '=' decoration
def get_display_title(title):
    length = len(title)
    str_output = '\n'
    str_output += get_display_multiple_characters('=', title, length)
    str_output += '\n'
    str_output += title
    str_output += '\n'
    str_output += get_display_multiple_characters('=', title, length)
    str_output += '\n\n'
    return str_output
    
# Display title, with dash decoration
def get_display_subTitle(title):
    length = len(title)
    str_output = '\n'
    str_output += title
    str_output += '\n'
    str_output += get_display_multiple_characters('-', title, length)
    str_output += '\n\n'
    return str_output

# Get display subtitle of current results, including number of results & confidence range
def get_display_current_results_subTitle(count_current_conf_results, current_conf_level):
    str_rules = 'rule'
    if(count_current_conf_results != 1):
        str_rules += 's'
    return get_display_subTitle(repr(count_current_conf_results) + " " + str_rules + " with confidence between " +  repr(confidences[current_conf_level]) + " and " + repr(confidences[current_conf_level + 1]))
    
# Display related results in a readable format: q1 ==> q2, with relevant statistics & math info.
def get_results_list(results_list):
    previous_conf_level = 4
    count_current_conf_results = 0
    str_results = get_display_title('Related Queries')
    str_items = ''
    
    for i in range(num_of_results):
        item = results_list[i]
        current_conf_level = item[5]
        if i > 0:
            previous_conf_level = results_list[i-1][5]

        if previous_conf_level != current_conf_level:
            str_results += get_display_current_results_subTitle(count_current_conf_results, current_conf_level)
            str_results += str_items
            str_items = ''
            count_current_conf_results = 0

        str_items += '{index:d}) {q1} ==> {q2}, conf={confidence:.3f}, #q1={q1_count}, #(q1 and q2)={combined_count}, similarity={similarity}\n\n'.format(index = count_current_conf_results + 1, q1 = item[0], q2 = item[1], confidence = item[4],  q1_count = item[3], combined_count = item[2], similarity = item[6])
        count_current_conf_results += 1

    str_results += get_display_current_results_subTitle(count_current_conf_results, current_conf_level - 1)
    str_results += str_items
    return str_results

In [13]:
if num_of_results < display_rules_num_of_records_threshold:
    results_list = [list(row) for row in results_df.collect()]
    print(get_results_list(results_list))
else:
    results_df.show(n2, truncate=False)


Related Queries


6 rules with confidence between 0.9 and 1
-----------------------------------------

1) undeground-love.com ==> dorki.ya-hoo.biz, conf=1.000, #q1=8, #(q1 and q2)=8, similarity=15

2) www.at ==> www.at&t.com, conf=1.000, #q1=9, #(q1 and q2)=9, similarity=6

3) diconary ==> dictionary, conf=1.000, #q1=7, #(q1 and q2)=7, similarity=2

4) vzwpix.com ==> viewpix.com, conf=1.000, #q1=7, #(q1 and q2)=7, similarity=2

5) www.friendspayday.com ==> www.friendspaydy.com, conf=1.000, #q1=7, #(q1 and q2)=7, similarity=1

6) localhookup.com ==> localhookupz.com, conf=1.000, #q1=6, #(q1 and q2)=6, similarity=1


4 rules with confidence between 0.8 and 0.9
-------------------------------------------

1) aol screen names ==> screen names, conf=0.875, #q1=8, #(q1 and q2)=7, similarity=4

2) letssingit ==> mycl.cravelyrics.com, conf=0.857, #q1=7, #(q1 and q2)=6, similarity=17

3) evo.qksrv.net ==> susan miller, conf=0.833, #q1=6, #(q1 and q2)=5, similarity=12

4) get out of debt planne

In [14]:
print("Free memory")
results_df.unpersist()

Free memory


DataFrame[query: string, query2: string, count_2_queries: bigint, count_query: bigint, conf: float, conf_level: int, similarity: int]

In [15]:
'''
    ======================================
    Interesting relations in the data set:
    ======================================
    
    1 interesting relation in confidence 1
    ---------------------------------------
    undeground-love.com ==> dorki.ya-hoo.biz, conf=1.000, #q1=8, #(q1 and q2)=8, similarity=15
    
    2 interesting relations in confidence between 0.8 and 0.9
    ---------------------------------------------------------
    letssingit ==> mycl.cravelyrics.com, conf=0.857, #q1=7, #(q1 and q2)=6, similarity=17
    evo.qksrv.net ==> susan miller, conf=0.833, #q1=6, #(q1 and q2)=5, similarity=12
    
    6 interesting relations in confidence between 0.6 and 0.8
    ----------------------------------------------------------
    elliot yamin ==> american idol
    harley davidson motorcycles ==> honda motorcycles, conf=0.667, #q1=9, #(q1 and q2)=6, similarity=12
    bombay kids ==> american idol, conf=0.625, #q1=8, #(q1 and q2)=5, similarity=10
    50 cent ==> ja rule, conf=0.667, #q1=6, #(q1 and q2)=4, similarity=6
    uranus ==> pluto, conf=0.667, #q1=6, #(q1 and q2)=4, similarity=6
    jupiter ==> uranus, conf=0.667, #q1=6, #(q1 and q2)=4, similarity=6
    
    There are no interestin relations in confidence between 0.9 and 1

'''

'\n    ======================================\n    Interesting relations in the data set:\n    ======================================\n    \n    1 interesting relation in confidence 1\n    ---------------------------------------\n    undeground-love.com ==> dorki.ya-hoo.biz, conf=1.000, #q1=8, #(q1 and q2)=8, similarity=15\n    \n    2 interesting relations in confidence between 0.8 and 0.9\n    ---------------------------------------------------------\n    letssingit ==> mycl.cravelyrics.com, conf=0.857, #q1=7, #(q1 and q2)=6, similarity=17\n    evo.qksrv.net ==> susan miller, conf=0.833, #q1=6, #(q1 and q2)=5, similarity=12\n    \n    6 interesting relations in confidence between 0.6 and 0.8\n    ----------------------------------------------------------\n    elliot yamin ==> american idol\n    harley davidson motorcycles ==> honda motorcycles, conf=0.667, #q1=9, #(q1 and q2)=6, similarity=12\n    bombay kids ==> american idol, conf=0.625, #q1=8, #(q1 and q2)=5, similarity=10\n    50